# Projet 2 : Accidents de la circulation en France (Spark)

#### Chloé Douarec, Solène Lesage - M2 Data Science, données biologiques et numériques - Novembre 2022

On s'intéresse à la base de données ONISR, pour les années 2011 à 2018, qui concerne les accidents de la circulation en France.
Cette base de données compte 4 rubriques : Caracteristiques, Lieux, Usagers, Vehicules.

- Lien vers l'archive contenant l'ensemble des données : https://coursspark.000webhostapp.com/data/accidents.zip 
- Informations sur la base et description : https://coursspark.000webhostapp.com/data/description-des-bases-de-donnees-onisr-annees-2005-a-2018.pdf

Dans cette partie, on souhaite calculer, parmi tous les véhicules impliqués dans un accident, et dont le chauffeur est un homme (resp. une femme), la proportion de ceux dont au moins l’un des occupants a été tué.

In [5]:
# pip install pyspark

In [6]:
from pyspark.sql import SparkSession
spark  = SparkSession.builder.master("local").appName('ProjetSpark').getOrCreate()

## Chargement des données

In [7]:
 # Listes des années
years = range(2011, 2019)

# Création d'une liste pour les fichiers csv
liste_usagers = []
for year in years:
    usagers = 'accidents/usagers_{}.csv'.format(year)
    liste_usagers.append(usagers)

print(liste_usagers)

['accidents/usagers_2011.csv', 'accidents/usagers_2012.csv', 'accidents/usagers_2013.csv', 'accidents/usagers_2014.csv', 'accidents/usagers_2015.csv', 'accidents/usagers_2016.csv', 'accidents/usagers_2017.csv', 'accidents/usagers_2018.csv']


On réalise une jointure des fichiers csv de toutes les années de 2011 à 2018 pour la rubrique Usagers, pour toutes les réunir en un seul dataframe.

In [8]:
import pandas as pd

# Initialisation de df_usagers : le premier .csv de usagers (2011)
df_usagers = spark.read.load(liste_usagers[0], format = 'csv',header=True) 

# On parcourt à compter de la deuxième année pour réaliser les jointures
for dataframe in liste_usagers[1:]:
    data_usagers = spark.read.load(dataframe, format = 'csv',header=True)                   # Chargement du fichier csv
    df_usagers = df_usagers.unionByName(data_usagers)                                       # Jointure

In [9]:
# Visualisation du dataframe
df_usagers.show(15)

+------------+-----+----+----+----+------+----+----+----+-----+-------+-------+
|     Num_Acc|place|catu|grav|sexe|trajet|secu|locp|actp|etatp|an_nais|num_veh|
+------------+-----+----+----+----+------+----+----+----+-----+-------+-------+
|201100000001|    1|   1|   3|   1|     5|  21|   0|   0|    0|   1995|    A01|
|201100000001|    1|   1|   1|   1|     5|  11|   0|   0|    0|   1949|    B02|
|201100000002|    1|   1|   1|   1|     0|  11|   0|   0|    0|   1967|    A01|
|201100000002|    1|   1|   3|   1|     0|  21|   0|   0|    0|   1963|    B02|
|201100000003|    1|   1|   1|   1|     1|  11|   0|   0|    0|   1989|    A01|
|201100000003| null|   3|   3|   2|     2|null|   4|   3|    1|   1993|    A01|
|201100000004|    1|   1|   4|   1|     5|  21|   0|   0|    0|   1971|    A01|
|201100000004|    1|   1|   3|   1|     5|  22|   0|   0|    0|   1980|    B02|
|201100000004|    1|   1|   1|   1|     5|  11|   0|   0|    0|   1980|    C03|
|201100000005|    1|   1|   1|   2|     

In [10]:
print("Il y a au total",df_usagers.count(),"usagers enregistrés dans des accidents de la circulation entre 2011 et 2018.")

Il y a au total 1078041 usagers enregistrés dans des accidents de la circulation entre 2011 et 2018.


### Ordonner/trier le dataframe selon les colonnes "Num_Acc", "num_veh" et "catu" par ordre croissant

Sachant que une ligne du dataframe df_usagers correspond à un usager, on a :
- Num_Acc : identifiant de l'accident concerné
- num_veh : identifiant du véhicule concerné parmi ceux de l'accident Num_Acc
- catu : catégorie de l'usager considéré. Prends les valeurs suivantes :\
       - 1 : Conducteur\
       - 2 : Passager\
       - 3 : Piéton\
       - 4 : Piéton en roller ou en trottinette

On souhaite ordonner les colonnes "Num_Acc", "num_veh" et "catu" par ordre croissant.\
Ordonner de cette façon le dataframe permet d'avoir les informations dans un ordre spécifique pour la suite du traitement.\
Pour chaque accidents ordonné par ordre croissant, on a tous les véhicules de l'accident par ordre alphabétique. Pour chacun de ces véhicules, on a tous les usagers associés les uns à la suite des autres, avec en premier le conducteur grâce au tri sur la variable catu.

In [11]:
df_usagers=df_usagers.sort("Num_Acc","num_veh","catu")

### Informations sur les conducteurs et quelques proportions intermédiaires

#### Tous les conducteurs

In [12]:
n_conduct = df_usagers.filter(df_usagers['catu'] == '1').count()
print("Nombre de conducteurs parmi les usagers impliqués dans un accident : ", n_conduct)

# Proportion de conducteurs sur l'ensemble des usagers : 74.43 %
prop_conducteurs=n_conduct*100/df_usagers.count() 
print("Proportion de conducteurs parmi tous les usagers impliqués dans un accident : %.3f" %prop_conducteurs,"%")

Nombre de conducteurs parmi les usagers impliqués dans un accident :  802357
Proportion de conducteurs parmi tous les usagers impliqués dans un accident : 74.427 %


#### Les conductrices femmes

A partir d'ici, on utilise une autre variable de la rubrique Usagers : la variable "sexe", qui indique le sexe de l'usager considéré. Elle prend les valeurs suivantes :    
- 1 : Homme
- 2 : Femme

In [13]:
# Nombre de conductrices parmi les usagers
n_conduct_f=df_usagers.filter((df_usagers['sexe'] == '2') &(df_usagers["catu"]=='1') ).count() # nombre d'usagers qui sont des femmes et qui étaient conductrices

# Proportion de conductrices parmi l'ensemble des conducteurs (hommes et femmes)
prop_conduct_f=n_conduct_f*100/n_conduct

print("Proportion de femmes parmi les conducteurs : %.3f" %prop_conduct_f,"%") # Proportion de femmes parmi les conducteurs : 25.88 %

Proportion de femmes parmi les conducteurs : 25.882 %


#### Les conducteurs hommes

In [14]:
# Nombre de conducteurs (hommes) parmi les usagers
n_conduct_h=df_usagers.filter((df_usagers['sexe'] == '1') &(df_usagers["catu"]=='1') ).count() # nombre d'usagers qui sont des hommes et qui étaient conducteurs

# Proportion de conducteurs (hommes) parmi l'ensemble des conducteurs (hommes et femmes)
prop_conduct_h=n_conduct_h*100/n_conduct
print("Proportion d'hommes parmi les conducteurs : %.3f" %prop_conduct_h,"%") # Proportion d'hommes parmi les conducteurs : 74.118 %

Proportion d'hommes parmi les conducteurs : 74.118 %


### Les usagers tués

On utilise la variable "grav", qui indique pour chaque usager son état suite à l'accident. Elle prend les valeurs suivantes : 
- 1 : Indemne
- 2 : Tué
- 3 : Blessé hospitalisé
- 4 : Blessé léger

In [15]:
df_usagers_tues = df_usagers.filter(df_usagers['grav'] == '2') # grav = 2 : usager tué

In [16]:
nombre_usagers=df_usagers.count()
nombre_tues=df_usagers_tues.count()

print("Parmi les",nombre_usagers,"usagers enregistrés,",nombre_tues,"ont été tués du fait de l'accident")

Parmi les 1078041 usagers enregistrés, 29200 ont été tués du fait de l'accident


### Ajout colonne sexe du conducteur dans pd_usagers

In [17]:
# Conversion du dataframe spark en dataframe pandas
pd_usagers=df_usagers.toPandas()

On ajoute la variable sexe_conduct, apportant l'information du sexe du conducteur du véhicule dans lequel se trouvait l'usager, au moment de l'accident.\
On calcule la valeur de cette variable à partir de la variable "sexe" et de la variable "catu". Plus précisement, le dataframe ayant été ordonné par accident, par véhicule, puis par la catégorie des usagers pour chaque véhicule, on sait que le premier usager décrit pour chaque véhicule est le conducteur. Cela est du au fait que la plus petite valeur de "catu" est 1, et que ça correspond au fait d'être le conducteur. 

#### Calcul des valeurs de sexe_conduct et ajout dans une liste

In [18]:
sexe_conduct=[]
for index, row in pd_usagers.iterrows():
    if row["catu"]=="1":
        sexe_conduct.append(row["sexe"])
        sexe_conducteur_courant=row["sexe"]
    else:
        sexe_conduct.append(sexe_conducteur_courant)

#### On affecte cette liste comme étant la variable "sexe_conduct" du dataframe pd_usagers

In [19]:
pd_usagers["sexe_conduct"]=sexe_conduct

In [20]:
pd_usagers.head(15)

,Num_Acc,place,catu,grav,sexe,trajet,secu,locp,actp,etatp,an_nais,num_veh,sexe_conduct
0,201100000001,1,1,3,1,5,21,0,0,0,1995,A01,1
1,201100000001,1,1,1,1,5,11,0,0,0,1949,B02,1
2,201100000002,1,1,1,1,0,11,0,0,0,1967,A01,1
3,201100000002,1,1,3,1,0,21,0,0,0,1963,B02,1
4,201100000003,1,1,1,1,1,11,0,0,0,1989,A01,1
5,201100000003,None,3,3,2,2,None,4,3,1,1993,A01,1
6,201100000004,1,1,4,1,5,21,0,0,0,1971,A01,1
7,201100000004,1,1,3,1,5,22,0,0,0,1980,B02,1
8,201100000004,1,1,1,1,5,11,0,0,0,1980,C03,1
9,201100000005,1,1,1,2,5,13,0,0,0,1980,A01,2


## Calcul de la proportion de véhicules dont au moins un des occupants a été tué, selon si le conducteur était un homme ou une femme

#### On sélectionne uniquement les usagers occupants de véhicules. Autrement dit, on retire les piétons du dataframe.

Cela revient à sélectionner les usagers dont la valeur pour la colonne "catu" est "1" ou "2", correspondant respectivement à "conducteur" et "passager" comme décrit plus tôt.

In [21]:
tableau_occupants=pd_usagers[(pd_usagers['catu']=='1') | (pd_usagers['catu']=='2')]

In [22]:
tableau_occupants[15:25] # Chaque ligne correspond à un occupant de véhicule

,Num_Acc,place,catu,grav,sexe,trajet,secu,locp,actp,etatp,an_nais,num_veh,sexe_conduct
17,201100000009,1,1,3,1,5,11,0,0,0,1963,A01,1
18,201100000010,1,1,1,1,1,13,0,0,0,1954,A01,1
20,201100000011,1,1,3,1,5,21,0,0,0,1961,A01,1
21,201100000012,1,1,3,1,5,11,0,0,0,1946,A01,1
22,201100000012,1,1,1,1,5,11,0,0,0,1936,B02,1
23,201100000013,1,1,1,1,0,11,0,0,0,1993,A01,1
24,201100000013,2,2,1,1,0,11,0,0,0,1992,A01,1
25,201100000013,2,2,1,1,0,11,0,0,0,1994,A01,1
26,201100000013,4,2,3,1,0,11,0,0,0,1991,A01,1
27,201100000014,1,1,3,1,5,21,0,0,0,1987,A01,1


Dans ce tableau, on trouve bien une ligne par occupant. En effet, on constate que le filtre sur "catu" = "1" ou "2" a bien fonctionné, puisqu'on trouve immédiatement des lignes où les occupants sont des conducteurs, et on repère un premier passager à la ligne 21.
On affiche le tableau seulement sur une portion contenant à la fois des conducteurs et des passagers pour illustrer.

### Tableau des véhicules triés selon le sexe du conducteur et la gravité de l'état des occupants

- grav = 1 : indemne
- grav = 2 : tué
- grav = 3 : blessé hospitalisé 
- grav = 4 : blessé léger

On récupère, pour chaque véhicule, la liste de l'état de gravité de ses occupants

In [23]:
tableau_listes=tableau_occupants.groupby(['sexe_conduct','Num_Acc','num_veh'])["grav"].agg(list)

In [24]:
pd.DataFrame(tableau_listes[15:25])

grav
sexe_conduct Num_Acc      num_veh              
1            201100000011 A01               [3]
             201100000012 A01               [3]
                          B02               [1]
             201100000013 A01      [1, 1, 1, 3]
             201100000014 A01            [3, 1]
                          B02            [3, 3]
             201100000015 A01               [1]
             201100000016 B02               [3]
             201100000017 B02               [3]
             201100000018 A01               [2]

#### Nombre de véhicules renseignés

In [25]:
total_vehicules_renseignes=tableau_listes[:].count() 
print(total_vehicules_renseignes)

803494


#### Nombre de véhicules ayant au moins un occupant tué

In [26]:
vehicules_usagers_tues=0
for gravites in tableau_listes:
    if '2' in gravites:
        vehicules_usagers_tues+=1

print(vehicules_usagers_tues,"véhicules ont des occupants qui ont été tués")

23254 véhicules ont des occupants qui ont été tués


#### Véhicules conduits par des hommes

In [27]:
pd.DataFrame(tableau_listes['1'])

grav
Num_Acc      num_veh     
201100000001 A01      [3]
             B02      [1]
201100000002 A01      [1]
             B02      [3]
201100000003 A01      [1]
...                   ...
201800057779 A01      [1]
201800057780 A01      [4]
201800057781 A01      [1]
201800057782 B01      [4]
201800057783 B01      [4]

[595564 rows x 1 columns]

In [28]:
n_veh_conduct_h=tableau_listes['1'].count()
print(n_veh_conduct_h, "véhicules sont conduits par des hommes")

595564 véhicules sont conduits par des hommes


Compte du nombre d'occupants tués lorsque le conducteur est un homme, et calcul de la proportion associée

In [29]:
liste_n_tues_conduct_h=[]

# On compte dans chaque liste de gravités le nombre de fois que '2' apparait
for gravites in tableau_listes['1']:
    compte=list(gravites).count('2')
    liste_n_tues_conduct_h.append(compte)

La proportion d'occupants tués lorsque le conducteur était un homme est donnée par la somme du nombre de personnes tuées lorsque le conducteur est un homme, multiplié par 100 et divisé par le nombre de véhicules conduits par un homme.

In [61]:
prop_tues_conduct_h=sum(liste_n_tues_conduct_h)*100/n_veh_conduct_h
print("%.3f"%prop_tues_conduct_h,"% des occupants ont été tués lorsque le conducteur était un homme")

3.515 % des occupants ont été tués lorsque le conducteur était un homme


In [31]:
print("Parmi les", n_veh_conduct_h, "véhicules conduits par des hommes, %.3f" %prop_tues_conduct_h ,"% sont liés au décès d'au moins un de leurs occupants, soit", liste_n_tues_conduct_h.count(1), "véhicules")

Parmi les 595564 véhicules conduits par des hommes, 3.515 % sont liés au décès d'au moins un de leurs occupants, soit 18434 véhicules


#### Véhicules conduits par des femmes

On procède de la même façon que pour les véhicules conduits par des hommes.

In [32]:
pd.DataFrame(tableau_listes['2'])

,,grav
Num_Acc,num_veh,
201100000005,A01,[1]
201100000007,A01,[1]
201100000016,A01,[1]
201100000017,A01,[1]
201100000020,A01,[1]
...,...,...
201800057769,A01,[3]
201800057776,B01,[4]
201800057778,B01,[1]


In [33]:
n_veh_conduct_f=tableau_listes['2'].count()
print(n_veh_conduct_f, "véhicules sont conduits par des femmes")

207930 véhicules sont conduits par des femmes


Compte du nombre d'occupants tués lorsque le conducteur est une femme, et calcul de la proportion associée

In [34]:
liste_n_tues_conduct_f=[]

# On compte dans chaque liste de gravités le nombre de fois que '2' apparait
for gravites in tableau_listes['2']:
    compte=list(gravites).count('2')
    liste_n_tues_conduct_f.append(compte)

In [62]:
prop_tues_conduct_f=sum(liste_n_tues_conduct_f)*100/n_veh_conduct_f
print("%.3f"%prop_tues_conduct_f,"% des occupants ont été tués lorsque le conducteur était une femme")

1.923 % des occupants ont été tués lorsque le conducteur était une femme


In [36]:
print("Parmi les", n_veh_conduct_f, "véhicules conduits par des femmes, %.3f" %prop_tues_conduct_f ,"% sont liés au décès d'au moins un de leurs occupants, soit",liste_n_tues_conduct_f.count(1), "véhicules")

Parmi les 207930 véhicules conduits par des femmes, 1.923 % sont liés au décès d'au moins un de leurs occupants, soit 3453 véhicules


## Première représentation - avec plotly.graph_objects

Le package plotly permet de réaliser des graphiques plus dynamiques, par exemple une représentation interactive ou un plus grand choix de personnalisation
- bibliothèque d'exemples : https://plotly.com/python/ 

On peut utiliser plotly.express, qui permet de réaliser rapidement des graphes, mais offre moins de possibilités de personnalisation que plotly.graph_objects

In [37]:
import plotly.graph_objects as go
import numpy as np

In [38]:
# Liste des genres
genre=["Femme","Homme"]
 
# Création du graphique
fig1 = go.Figure( data=[go.Bar(x=genre, y=[prop_tues_conduct_f,prop_tues_conduct_h])] )

# Paramètres des colonnes
fig1.update_traces(
        marker_color=["rgb(160,97,119)","rgb(128,177,211)"],    # couleur des colonnes
        marker_line_color="white",                     # couleur du contour des colonnes
        marker_line_width=2,                           # épaisseur du contour des colonnes
        opacity=0.7,                                   # opacité des colonnes (compris entre 0 et 1)
        )

# Paramètres de la fenêtre graphique et des axes
fig1.update_layout(
    title_text='Proportion de véhicules dont au moins un des occupants a été tué, selon le sexe du conducteur',
    margin=dict(l=50, r=20, t=60, b=10), # marges du graphique
    paper_bgcolor="#f2f5f9",             # couleur de fond
    width=900,                           # largeur du graphique
    height=600,                          # hauteur du graphique
    bargap=0.55,                          # écart entre les colonnes

    yaxis=dict(
        title_text="Proportion (%)",
        tickvals=np.arange(0,4,0.2),    # valeurs pour l'axe y et graduation
        titlefont=dict(size=15),
    ),
    xaxis=dict(
        title_text="Sexe du conducteur",
        titlefont=dict(size=15),
    )
)

# Affichage du graphique
fig1.show()

Ce premier graphique nous montre que la proportion de véhicules dont au moins un des occupants a été tué, lorsque les conducteurs étaient des hommes, est presque 2 fois plus grande que lorsque ce sont des femmes qui conduisaient.\
Les accidents semblent donc moins mortels lorsque ce sont des femmes qui conduisaient, sur la période 2011-2018.\
Pour avoir plus de détails sur ces différences et voir si le propos peut être nuancé, on propose de regarder les comptes exacts de personnes tués par véhicules, au lieu de seulement regarder les proportions de véhicules dont au moins 1 occupant a été tué.

## Graphique de la proportion de véhicules dont un certain nombre d'occupants ont été tués, selon le sexe du conducteur

### Mise en forme des données pour le graphique

Compte du nombre de véhicules conduit par un homme/une femme dont 1/2/3 occupants ont été tués

In [39]:
femmes1=liste_n_tues_conduct_f.count(1)
femmes2=liste_n_tues_conduct_f.count(2)
femmes3=liste_n_tues_conduct_f.count(3)

hommes1=liste_n_tues_conduct_h.count(1)
hommes2=liste_n_tues_conduct_h.count(2)
hommes3=liste_n_tues_conduct_h.count(3)

Compte du nombre de véhicules conduit par un homme/une femme dont 4 occupants ou plus ont été tués

In [40]:
k=0
for i in liste_n_tues_conduct_f:
    if i >=4: k+=1
femmes4=k

In [41]:
k=0
for i in liste_n_tues_conduct_h:
    if i >=4: k+=1

hommes4=k

Tableau avec le nombre de véhicules concernés par catégorie

In [42]:
df_exact = pd.DataFrame({'Femmes': [femmes1,femmes2,femmes3,femmes4], 'Hommes': [hommes1,hommes2,hommes3,hommes4]}, index=['1','2','3','4+'])
df_exact

,Femmes,Hommes
1,3453,18434
2,221,945
3,23,130
4+,7,41


Tableau avec la proportion de véhicules concernés par catégorie

In [43]:
df_exact["Femmes"]=df_exact["Femmes"]*100/n_veh_conduct_f
df_exact["Hommes"]=df_exact["Hommes"]*100/n_veh_conduct_h

In [44]:
df_exact

,Femmes,Hommes
1,1.660655,3.095217
2,0.106286,0.158673
3,0.011061,0.021828
4+,0.003367,0.006884


In [45]:
niveaux=['1','2','3','4+']

fig2 = go.Figure()
fig2.add_trace( go.Bar(name='Femmes', x=niveaux, y=df_exact["Femmes"], marker_color="#AC9FBB") )
fig2.add_trace( go.Bar(name='Hommes', x=niveaux, y=df_exact["Hommes"], marker_color="#033F63") )

# Paramètres des colonnes
fig2.update_traces(
            marker_line_color='rgb(190,186,218)',   # couleur du contour des colonnes
            marker_line_width=1.5,                  # épaisseur du contour des colonnes
            opacity=0.8                             # opacité des colonnes (compris entre 0 et 1)
        )

# Paramètres de la fenêtre graphique et des axes
fig2.update_layout(
    barmode='group',
    title_text="Proportion de véhicules dont un certain nombre d'occupants ont été tués, selon le sexe du conducteur",
    margin=dict(l=50, r=20, t=60, b=10), # marges du graphique
    paper_bgcolor="#f2f5f9",             # couleur de fond
    width=1000,                          # largeur du graphique
    height=600,
    bargap=0.4,                          # écart entre les colonnes

    yaxis=dict(
        title_text="Proportion de véhicules (%)",
        tickvals=np.arange(0,3.3,0.10),    # valeurs pour l'axe y et graduation
        titlefont=dict(size=15),
    ),
    xaxis=dict(
        title_text="Nombre d'occupants tués",
        titlefont=dict(size=15),
    )
)
fig2.show()

Les graphes plotly sont des graphes interactifs. En particulier, on peut sélectionner une zone particulière du graphique que l'on souhaite mieux voir en cliquant sur "box select". Par exemple ici, c'est très utile pour mieux visualiser les colonnes concernant les véhicules dont 3 occupants ont été tués, et 4 et plus, étant donné que les valeurs de proportions sont très petites, par rapport à celles obtenues pour les véhicules ayant eu un seul occupant tué. Ensuite, une fois la zone sélectionné, on peut facilement observer les valeurs exactes des colonnes concernées, en passant la souris dessus.

Ce second graphique montre qu'une majorité des véhicules dont des occupants ont été tués, sont en fait des véhicules où il n'y a eu qu'un seul occupant tué du fait de l'accident. Pour les véhicules où il y a eu deux occupants tués, cela représente seulement 0.11% des véhicules accidentés conduits par une femme, et 0.16% de ceux conduits par un homme.\
Ce graphique nous montre également que la différence observée entre les conducteurs hommes et femmes se maintient, lorsqu'on regarde plus en détail les différences selon le nombre exact d'occupants tués, bien que l'écart est très atténué à partir de 3 et plus.
On peut donc considérer qu'à compter de 3 occupants et plus tués, le sexe du conducteur a beaucoup moins d'influence. On peut supposer que ces accidents sont ceux étant les plus graves, et donc que le sexe du conducteur aurait peu d'influence sur ces accidents très mortels, s'ils sont du à une autre cause extérieure que le conducteur en premier lieu.

On peut donc déduire des deux graphiques que les accidents semblent plus mortels lorsque ce sont des hommes qui conduisent, plutôt que des femmes. Néanmoins, on garde en tête que pour les accidents les plus mortels, la différence est plus faible.

## Calcul de la proportion d'occupants tués, selon s'ils étaient conducteurs ou passagers, en prenant en compte le sexe du conducteur 

On propose de regarder la différence de proportions de personnes tuées, selon qu'ils étaient conducteurs ou passagers, toujours en fonction du sexe du conducteur. Cela nous permettra éventuellement de nuancer les résultats précédents, et d'observer quel type d'occupants sont les plus tués, entre les conducteurs eux-mêmes et leurs passagers.

Pour cela, on reprend le tableau "tableau_occupants" représentant tous les occupants de véhicules, avec l'ensemble des variables traitées précédemment pour les premiers graphiques. On va considérer en plus la variable "catu", qui va permettre de distinguer les occupants, selon s'ils étaient conducteur ou passager.

In [46]:
tableau_occupants.head()

,Num_Acc,place,catu,grav,sexe,trajet,secu,locp,actp,etatp,an_nais,num_veh,sexe_conduct
0,201100000001,1,1,3,1,5,21,0,0,0,1995,A01,1
1,201100000001,1,1,1,1,5,11,0,0,0,1949,B02,1
2,201100000002,1,1,1,1,0,11,0,0,0,1967,A01,1
3,201100000002,1,1,3,1,0,21,0,0,0,1963,B02,1
4,201100000003,1,1,1,1,1,11,0,0,0,1989,A01,1


On procède de la même façon que précédemment, avec un groupby permettant de créer une liste des gravités, en ajoutant la différenciation des occupants selon la valeur de "catu", c'est-à-dire entre conducteurs ("catu"=1) et passagers ("catu"="2") tel que nous venons de l'expliquer. On compare toujours les résultats selon le sexe du conducteur.

In [47]:
sep_conduct_passager=tableau_occupants.groupby(['sexe_conduct','Num_Acc','num_veh','catu'])["grav"].agg(list)

In [48]:
pd.DataFrame(sep_conduct_passager[15:25])

grav
sexe_conduct Num_Acc      num_veh catu           
1            201100000011 A01     1           [3]
             201100000012 A01     1           [3]
                          B02     1           [1]
             201100000013 A01     1           [1]
                                  2     [1, 1, 3]
             201100000014 A01     1           [3]
                                  2           [1]
                          B02     1           [3]
                                  2           [3]
             201100000015 A01     1           [1]

On récupère séparemment la partie où "sexe_conduct" vaut 1 (conduteurs hommes), et celle valant 2 (condutrices femmes).

In [49]:
sep_conduct_passager_h=sep_conduct_passager['1'] # Tableau des cas où le conducteur est un homme
sep_conduct_passager_f=sep_conduct_passager['2'] # Tableau des cas où le conducteur est une femme

On transforme le tableau sous forme de "groupby" en vrai tableau, sans cases vides dûes à la commande groupby.

In [50]:
sep_conduct_passager_h=sep_conduct_passager_h.reset_index()
sep_conduct_passager_f=sep_conduct_passager_f.reset_index()

#### Cas conducteurs

In [51]:
conduct_h_conduct_tues=[]       # Conducteurs hommes
conduct_h_passagers_tues=[]     # Passagers dont le conducteur est homme

# On parcourt le dataframe
for index,row in sep_conduct_passager_h.iterrows():
    if row["catu"]=='1': 
        # Si c'est un conducteur
        compte=list(row["grav"]).count('2') 
        conduct_h_conduct_tues.append(compte)
        
    elif row["catu"]=='2': 
        # Si c'est un passager
        compte=list(row["grav"]).count('2') 
        conduct_h_passagers_tues.append(compte)

Proportion de conducteurs tués lorsque celui-ci était un homme

In [52]:
prop_conduct_h_c_tues=sum(conduct_h_conduct_tues)*100/n_veh_conduct_h # n_veh_conduct_h a été calculé auparavant

print("%.3f"%prop_conduct_h_c_tues,"% des conducteurs ont été tués sachant qu'ils étaient des hommes")

2.880 % des conducteurs ont été tués sachant qu'ils étaient des hommes


Proportion de passagers tués lorsque le conducteur était un homme

In [53]:
prop_conduct_h_p_tues=sum(conduct_h_passagers_tues)*100/n_veh_conduct_h # n_veh_conduct_h a été calculé auparavant

print("%.4f"%prop_conduct_h_p_tues,"% des passagers ont été tués lorsque le conducteur du véhicule était un homme")

0.6349 % des passagers ont été tués lorsque le conducteur du véhicule était un homme


#### Cas conductrices

In [54]:
conduct_f_conduct_tues=[]   # Conducteurs femmes
conduct_f_passagers_tues=[] # Passagers de véhicules dont le conducteur était une femme

# On parcourt le dataframe
for index,row in sep_conduct_passager_f.iterrows():
    if row["catu"]=='1':
        # Si c'est une conductrice
        compte=list(row["grav"]).count('2') 
        conduct_f_conduct_tues.append(compte)

    elif row["catu"]=='2': 
        # Si c'est un passager
        compte=list(row["grav"]).count('2') 
        conduct_f_passagers_tues.append(compte)

Proportion de conductrices tuées

In [55]:
prop_conduct_f_c_tues=sum(conduct_f_conduct_tues)*100/n_veh_conduct_f # n_veh_conduct_f a été calculé auparavant

print("%.4f"%prop_conduct_f_c_tues,"% des conductrices ont été tuées")

1.4760 % des conductrices ont été tuées


Proportion de passagers tués lorsque le conducteur était une femme

In [56]:
prop_conduct_f_p_tues=sum(conduct_f_passagers_tues)*100/n_veh_conduct_f # n_veh_conduct_f a été calculé auparavant

print("%.4f"%prop_conduct_f_p_tues,"% des passagers ont été tués lorsque le conducteur du véhicule était une femme")

0.4468 % des passagers ont été tués lorsque le conducteur du véhicule était une femme


## Graphique de la proportion d'occupants tués, selon s'ils étaient conducteurs ou passagers, en prenant en compte le sexe du conducteur 

On met sous forme d'un dataframe les proportions que l'on vient de calculer, dans le but de réaliser un graphique. 

In [57]:
df_conduct_passagers = pd.DataFrame({'Conductrice (F)': [prop_conduct_f_c_tues,prop_conduct_f_p_tues], 'Conducteur (H)': [prop_conduct_h_c_tues,prop_conduct_h_p_tues]}, index=['Conducteurs tués','Passagers tués'])

In [58]:
df_conduct_passagers

,Conductrice (F),Conducteur (H)
Conducteurs tués,1.475977,2.879791
Passagers tués,0.446785,0.634860


Réalisation du graphique

In [59]:
place=["Conducteur","Passager"]

# Création de la figure
fig3 = go.Figure()

# Colonnes avec conducteurs hommes
fig3.add_trace( go.Bar(name='Conducteur (H)', x=place, y=df_conduct_passagers["Conducteur (H)"], marker_color="darkslateblue") )
# Colonnes avec conductrices
fig3.add_trace( go.Bar(name='Conductrice (F)', x=place, y=df_conduct_passagers["Conductrice (F)"], marker_color="darkorange") )


# Paramètres des colonnes
fig3.update_traces(
            marker_line_color='white',        # couleur du contour des colonnes
            marker_line_width=2,              # épaisseur du contour des colonnes
            opacity=0.75                      # opacité des colonnes (compris entre 0 et 1)
        )

# Paramètres de la fenêtre graphique et des axes
fig3.update_layout(
    barmode='group',
    title_text="Proportion de conducteurs (resp. passagers) tués, selon le sexe du conducteur",
    margin=dict(l=50, r=20, t=60, b=10),  # marges du graphique
    paper_bgcolor="#f2f5f9",              # couleur de fond
    width=1000,                           # largeur du graphique
    height=600,                           # hauteur du graphique
    bargap=0.5,                           # écart entre les colonnes

    yaxis=dict(
        title_text="Proportion (%)",
        tickvals=np.arange(0,4.5,0.15),   # graduations de l'axe y
        titlefont=dict(size=15),
    ),
    xaxis=dict(
        title_text="Place de l'occupant",
        titlefont=dict(size=15),
    ))

fig3.show()

Ce graphique mets en avant qu'il y a plus de conducteurs tués que de passagers en terme de proportion.
On voit aussi qu'il y a un maintient de l'écart entre les hommes et les femmes concernant les proportions de conducteurs tués : les conductrices sont moins tuées que les conducteurs. Parcontre, on constate que concernant les passagers tués, la différence est moins importante selon que le conducteur soit un homme ou une femme, en comparaison avec les conducteurs tués, puisqu'on obtient une différence de seulement environ 0.18% contre 1.40% de différence pour les conducteurs.

Ce graphique permet donc de compléter les informations obtenues avec les précédents, et nous permet de nuancer le propos concernant les différences en termes de proportions d'occupants tués, entre les conducteurs hommes et les conductrices. On conclut donc que de manière générale, les accidents semblent plus mortels lorsque le conducteur est un homme plutôt qu'une femme. Néanmoins, pour les accidents les plus mortels pour les occupants de véhicules (à compter de 3 occupants tués et plus), la différence est peu marquée entre les conducteurs hommes et femmes. De plus, on constate que généralement, il y a plus de conducteurs tués que de passagers tués en terme de proportion.